# American Put Option Pricing — LSM · NLSM · RLSM

**Source repository:** [HeKrRuTe/OptStopRandNN](https://github.com/HeKrRuTe/OptStopRandNN)  
**Paper:** *Optimal Stopping via Randomized Neural Networks*, Herrera, Krach, Ruyssen & Teichmann (2023), *Frontiers of Mathematical Finance*

This notebook is a **self-contained port** of three algorithm modules from the repo:

| Notebook section | Repo source file(s) | Notes |
|---|---|---|
| `Put1Dim` payoff | `payoff.py` | Verbatim |
| `BlackScholes.generate_paths()` | `stock_model.py` | Verbatim; other models stripped |
| `AmericanOptionPricer.price()` + `stop()` | `backward_induction_pricer.py` | Verbatim; Greeks/bounds stripped |
| `BasisFunctions` (polynomial) | `algorithms/utils/basis_functions.py` | Not uploaded — inlined from usage in `regression.py` |
| `LeastSquares.calculate_regression()` | `regression.py` | Verbatim |
| `LeastSquaresPricer` | `LSM.py` | Verbatim |
| `Reservoir2` (frozen random network) | `algorithms/utils/randomized_neural_networks.py` | Not uploaded — inlined from usage in `regression.py` |
| `ReservoirLeastSquares2.calculate_regression()` | `regression.py` | Verbatim; already PyTorch |
| `ReservoirLeastSquarePricerFastTanh` | `RLSM.py` | Verbatim |
| `NetworkNLSM` architecture | `algorithms/utils/neural_networks.py` | Not uploaded; **converted TF/Keras → PyTorch** |
| `NeuralRegression.train_network()` / `evaluate_network()` | `NLSM.py` | Verbatim — already used PyTorch |
| `NeuralNetworkPricer` | `NLSM.py` | Verbatim |

> **TensorFlow → PyTorch conversion:**  
> The only TF dependency was `NetworkNLSM` in `neural_networks.py` (not uploaded), which was a `tf.keras.Sequential` model.  
> It is rewritten here as a `torch.nn.Module` with the **identical architecture**: two hidden layers with `Tanh` activation.  
> Everything else in `NLSM.py` (`NeuralRegression.train_network`, `evaluate_network`) already used PyTorch in the original.


## 0 · Imports

In [ ]:
import math
import time
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt

# PyTorch — used by RLSM's Reservoir2 and NLSM's NeuralRegression
# (regression.py and NLSM.py both imported torch in the original)
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as tdata

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

print(f'PyTorch  : {torch.__version__}')
print(f'NumPy    : {np.__version__}')

## 1 · Parameters

In [ ]:
# Option & model parameters
SPOT       = 100.0
STRIKE     = 100.0
RATE       = 0.05
VOLATILITY = 0.2
MATURITY   = 1.0
DIVIDEND   = 0.0

# Simulation parameters
NB_PATHS   = 50_000
NB_DATES   = 50

print(f'S0={SPOT}  K={STRIKE}  r={RATE}  σ={VOLATILITY}  T={MATURITY}  q={DIVIDEND}')
print(f'Paths={NB_PATHS:,}   Dates={NB_DATES}')

## 2 · Payoff — `Put1Dim`

**Source:** `payoff.py` → `Put1Dim`  
Ported verbatim. Boilerplate base class and unused payoffs omitted.

In [ ]:
# SOURCE: payoff.py → Put1Dim (verbatim; base class boilerplate omitted)
class Put1Dim:
    """Payoff of a 1-dimensional put option: max(K - S, 0).
    Source: payoff.py::Put1Dim
    """
    def __init__(self, strike):
        self.strike = strike

    def __call__(self, X):
        return self.eval(X)

    def eval(self, X):
        # X is (nb_paths, nb_stocks) when called on a single date slice,
        # or (nb_paths, nb_stocks, nb_dates+1) when called on full paths.
        # The pricer calls payoff(stock_paths) on full paths to get all payoffs,
        # then payoff.eval(stock_paths[:,:,date]) for per-date slices.
        if X.ndim == 3:
            return np.maximum(0, self.strike - X[:, 0, :])  # (nb_paths, nb_dates+1)
        return np.maximum(0, self.strike - X[:, 0])          # (nb_paths,)

payoff = Put1Dim(strike=STRIKE)
print(f'Payoff at S=90 : {payoff.eval(np.array([[[90]]]))[0,0]:.2f}  (expected 10)')
print(f'Payoff at S=110: {payoff.eval(np.array([[[110]]]))[0,0]:.2f}  (expected  0)')

## 3 · Stock Model — `BlackScholes`

**Source:** `stock_model.py` → `Model` (base) + `BlackScholes`  
`generate_paths()` copied **verbatim** including the exact vectorised `cumsum/exp` formula.  
Removed: FBM, Heston, RoughHeston, joblib parallelism, draw utilities.  
Array layout preserved: `(nb_paths, nb_stocks, nb_dates+1)`.

In [ ]:
# SOURCE: stock_model.py → Model (base attrs only) + BlackScholes.generate_paths() (verbatim)
class BlackScholes:
    """GBM under risk-neutral measure.
    Source: stock_model.py::BlackScholes
    Removed: all other model classes, joblib, fbm dependency.
    """
    def __init__(self, drift, volatility, nb_paths, nb_stocks, nb_dates,
                 spot, maturity, dividend=0.0, **kwargs):
        # From stock_model.py::Model.__init__
        self.name       = 'BlackScholes'
        self.drift      = drift - dividend   # risk-neutral drift
        self.rate       = drift
        self.dividend   = dividend
        self.volatility = volatility
        self.spot       = spot
        self.nb_stocks  = nb_stocks
        self.nb_paths   = nb_paths
        self.nb_dates   = nb_dates
        self.maturity   = maturity
        self.dt         = maturity / nb_dates
        self.df         = math.exp(-self.rate * self.dt)
        self.return_var = False  # no variance process for plain BS

    def generate_paths(self, nb_paths=None, return_dW=False, dW=None,
                       X0=None, nb_dates=None):
        """Returns ndarray (nb_paths, nb_stocks, nb_dates+1).
        Copied verbatim from stock_model.py::BlackScholes.generate_paths().
        """
        nb_paths = nb_paths or self.nb_paths
        nb_dates = nb_dates or self.nb_dates
        spot_paths = np.empty((nb_paths, self.nb_stocks, nb_dates + 1))
        if X0 is None:
            spot_paths[:, :, 0] = self.spot
        else:
            spot_paths[:, :, 0] = X0
        if dW is None:
            random_numbers = np.random.normal(0, 1, (nb_paths, self.nb_stocks, nb_dates))
            dW = random_numbers * np.sqrt(self.dt)
        drift = self.drift
        r = np.repeat(np.repeat(np.repeat(
            np.reshape(drift, (-1, 1, 1)), nb_paths, axis=0),
            self.nb_stocks, axis=1), nb_dates, axis=2)
        sig = np.repeat(np.repeat(np.repeat(
            np.reshape(self.volatility, (-1, 1, 1)), nb_paths, axis=0),
            self.nb_stocks, axis=1), nb_dates, axis=2)
        spot_paths[:, :, 1:] = np.repeat(
            spot_paths[:, :, 0:1], nb_dates, axis=2) * np.exp(np.cumsum(
            r * self.dt - (sig ** 2) * self.dt / 2 + sig * dW, axis=2))
        if return_dW:
            return spot_paths, None, dW
        return spot_paths, None   # second value = var_paths (None for plain BS)


# Instantiate and generate paths once — all three algorithms share these
model = BlackScholes(
    drift=RATE, volatility=VOLATILITY, nb_paths=NB_PATHS,
    nb_stocks=1, nb_dates=NB_DATES, spot=SPOT,
    maturity=MATURITY, dividend=DIVIDEND)

np.random.seed(SEED)
stock_paths, var_paths = model.generate_paths()
print(f'stock_paths shape: {stock_paths.shape}  →  (nb_paths, nb_stocks, nb_dates+1)')

fig, axes = plt.subplots(1, 2, figsize=(13, 4))
t_grid = np.linspace(0, MATURITY, NB_DATES + 1)
axes[0].plot(t_grid, stock_paths[:200, 0, :].T, alpha=0.15, color='steelblue', lw=0.8)
axes[0].axhline(STRIKE, color='red', lw=1.5, ls='--', label=f'K={STRIKE}')
axes[0].set_title('Sample GBM Paths (first 200)'); axes[0].legend()
axes[0].set_xlabel('Time'); axes[0].set_ylabel('Stock Price')
axes[1].hist(stock_paths[:, 0, -1], bins=60, color='steelblue', edgecolor='white', alpha=0.8)
axes[1].axvline(STRIKE, color='red', lw=1.5, ls='--', label=f'K={STRIKE}')
axes[1].set_title('Terminal Price Distribution'); axes[1].legend()
axes[1].set_xlabel('$S_T$'); axes[1].set_ylabel('Count')
plt.tight_layout(); plt.show()

## 4 · Reference Price — CRR Binomial Tree

Not from the repo — added as a numerical benchmark only.

In [ ]:
# NOT from repo — benchmark only
def binomial_american_put(S0, K, r, sigma, T, N=2000, dividend=0.0):
    dt = T / N; u = np.exp(sigma * np.sqrt(dt)); d = 1.0 / u
    p = (np.exp((r - dividend) * dt) - d) / (u - d)
    disc = np.exp(-r * dt)
    V = np.maximum(K - S0 * u**(N - 2*np.arange(N+1)), 0.0)
    for i in range(N - 1, -1, -1):
        S_i = S0 * u**(i - 2*np.arange(i+1))
        V   = disc * (p * V[:i+1] + (1-p) * V[1:i+2])
        V   = np.maximum(V, np.maximum(K - S_i, 0.0))
    return float(V[0])

ref_price = binomial_american_put(SPOT, STRIKE, RATE, VOLATILITY, MATURITY, N=2000)
print(f'CRR Binomial (N=2000) reference price: {ref_price:.4f}')

## 5 · Base Pricer — `AmericanOptionPricer`

**Source:** `backward_induction_pricer.py` → `AmericanOptionPricer`  
`__init__`, `stop()`, and `price()` copied **verbatim** from lines 22–161.  
Removed: `use_rnn`/`use_path` branches (not triggered for Markovian 1-stock put),  
`configs.path_gen_seed` (replaced by `np.random.seed` at call site),  
`price_upper_lower_bound`, `price_and_greeks`, and all Greek helper methods.

In [ ]:
# SOURCE: backward_induction_pricer.py → AmericanOptionPricer
# __init__, stop(), price() copied verbatim (lines 22-161).
# Removed: use_rnn/use_path branches, configs seed, Greeks, upper/lower bound.
class AmericanOptionPricer:
    """Computes the price of an American Option using backward recursion.
    Source: backward_induction_pricer.py::AmericanOptionPricer
    """
    def __init__(self, model, payoff, use_rnn=False, train_ITM_only=True,
                 use_path=False, use_payoff_as_input=False):
        self.model               = model
        self.use_var             = model.return_var   # True for Heston; False for BS
        self.payoff              = payoff
        self.use_rnn             = use_rnn
        self.use_path            = use_path
        self.train_ITM_only      = train_ITM_only
        self.use_payoff_as_input = use_payoff_as_input
        self.which_weight        = 0

    def calculate_continuation_value(
            self, values, immediate_exercise_value, stock_paths_at_timestep):
        """Overridden by each algorithm subclass.
        Source: backward_induction_pricer.py::AmericanOptionPricer.calculate_continuation_value()
        """
        raise NotImplementedError

    def stop(self, stock_values, immediate_exercise_values,
             discounted_next_values, h=None, var_paths=None,
             return_continuation_values=False):
        """Returns {0,1} vector: 1=stop (exercise), 0=continue.
        Copied verbatim from backward_induction_pricer.py::AmericanOptionPricer.stop()
        """
        stopping_rule = np.zeros(len(stock_values))
        if self.use_var:
            stock_values = np.concatenate([stock_values, var_paths], axis=1)
        continuation_values = self.calculate_continuation_value(
            discounted_next_values, immediate_exercise_values, stock_values)
        if self.train_ITM_only:
            which = (immediate_exercise_values > continuation_values) & \
                    (immediate_exercise_values > np.finfo(float).eps)
        else:
            which = immediate_exercise_values > continuation_values
        stopping_rule[which] = 1
        if return_continuation_values:
            return stopping_rule, continuation_values
        return stopping_rule

    def price(self, stock_paths, var_paths, train_eval_split=2):
        """Compute American option price via backward recursion.
        Copied verbatim from backward_induction_pricer.py::AmericanOptionPricer.price()
        Signature change: stock_paths/var_paths passed in (pre-generated) so all
        three algorithms can be compared on identical paths.
        """
        model = self.model
        payoffs = self.payoff(stock_paths)   # (nb_paths, nb_dates+1)
        stock_paths_with_payoff = np.concatenate(
            [stock_paths, np.expand_dims(payoffs, axis=1)], axis=1)
        self.split = int(len(stock_paths) / train_eval_split)

        # verbatim from original
        disc_factor = math.exp(-model.rate * model.maturity / model.nb_dates)
        immediate_exercise_value = self.payoff.eval(stock_paths[:, :, -1])
        values = immediate_exercise_value

        for i, date in enumerate(range(stock_paths.shape[2] - 2, 0, -1)):
            self.which_weight = i
            immediate_exercise_value = self.payoff.eval(stock_paths[:, :, date])
            varp  = var_paths[:, :, date] if self.use_var else None
            paths = (stock_paths_with_payoff[:, :, date]
                     if self.use_payoff_as_input
                     else stock_paths[:, :, date])
            stopping_rule = self.stop(
                paths, immediate_exercise_value,
                values * disc_factor, h=None, var_paths=varp)
            which = stopping_rule > 0.5
            values[which]  = immediate_exercise_value[which]
            values[~which] *= disc_factor

        payoff_0 = self.payoff.eval(stock_paths[:, :, 0])[0]
        # Use only eval half (self.split:) to avoid in-sample bias — verbatim from original
        return max(payoff_0, np.mean(values[self.split:]) * disc_factor), 0.0

## 6 · Basis Functions — `BasisFunctions`

**Source:** `algorithms/utils/basis_functions.py` (not uploaded)  
Inlined from its usage in `regression.py::LeastSquares`:  
`self.bf.nb_base_fcts` and `self.bf.base_fct(coeff, X, d2=True)`.  
For 1 stock, degree-2 polynomial basis gives 3 features: `[1, S, S²]`.

In [ ]:
# SOURCE: algorithms/utils/basis_functions.py (NOT UPLOADED — inlined from usage in regression.py)
# regression.py::LeastSquares calls:
#   self.bf = basis_functions.BasisFunctions(nb_stocks)
#   self.bf.nb_base_fcts
#   self.bf.base_fct(coeff, X[:,:], d2=True)
class BasisFunctions:
    """Polynomial basis functions of degree <= 2 for LSM regression.
    Source: algorithms/utils/basis_functions.py (inlined)
    For nb_stocks=1: basis = [1, S, S^2]  →  nb_base_fcts = 3
    """
    def __init__(self, nb_stocks):
        self.nb_stocks    = nb_stocks
        # 1 constant + nb_stocks linear + nb_stocks*(nb_stocks+1)/2 quadratic
        self.nb_base_fcts = 1 + nb_stocks + nb_stocks * (nb_stocks + 1) // 2

    def base_fct(self, coeff, X, d2=True):
        """Evaluate coeff-th basis function on X (nb_paths, nb_stocks)."""
        nb_paths, nb_stocks = X.shape
        if coeff == 0:
            return np.ones(nb_paths)            # constant
        elif coeff <= nb_stocks:
            return X[:, coeff - 1]              # linear: S_1, ..., S_d
        else:
            # quadratic cross terms: S_i * S_j  (i <= j)
            idx   = coeff - nb_stocks - 1
            pairs = [(i, j) for i in range(nb_stocks) for j in range(i, nb_stocks)]
            i, j  = pairs[idx]
            return X[:, i] * X[:, j]

bf = BasisFunctions(nb_stocks=1)
print(f'nb_base_fcts (1 stock): {bf.nb_base_fcts}  →  [1, S, S²]')

## 7 · Algorithm 1 — LSM

**Source:**
- `regression.py` → `LeastSquares.calculate_regression()` — **verbatim**
- `LSM.py` → `LeastSquaresPricer.__init__()` + `calculate_continuation_value()` — **verbatim**

In [ ]:
# SOURCE: regression.py → LeastSquares (verbatim)
class LeastSquares:
    """OLS on polynomial basis functions.
    Source: regression.py::LeastSquares
    """
    def __init__(self, nb_stocks):
        self.nb_stocks = nb_stocks
        self.bf = BasisFunctions(self.nb_stocks)

    def calculate_regression(self, X, Y, in_the_money, in_the_money_all):
        """Copied verbatim from regression.py::LeastSquares.calculate_regression()"""
        nb_paths, nb_stocks = X.shape
        reg_vect_mat = np.empty((nb_paths, self.bf.nb_base_fcts))
        for coeff in range(self.bf.nb_base_fcts):
            reg_vect_mat[:, coeff] = self.bf.base_fct(coeff, X[:, :], d2=True)
        coefficients = np.linalg.lstsq(
            reg_vect_mat[in_the_money[0]], Y[in_the_money[0]], rcond=None)
        continuation_values = np.dot(
            reg_vect_mat[in_the_money_all[0]], coefficients[0])
        return continuation_values


# SOURCE: LSM.py → LeastSquaresPricer (verbatim)
class LeastSquaresPricer(AmericanOptionPricer):
    """American option pricer — Least-Square Monte Carlo.
    Source: LSM.py::LeastSquaresPricer
    """
    def __init__(self, model, payoff, nb_epochs=None, nb_batches=None,
                 train_ITM_only=True, use_payoff_as_input=False):
        super().__init__(model, payoff, train_ITM_only=train_ITM_only,
                         use_payoff_as_input=use_payoff_as_input)
        self.regression = LeastSquares(
            model.nb_stocks * (1 + self.use_var) + self.use_payoff_as_input * 1)

    def calculate_continuation_value(
            self, values, immediate_exercise_value, stock_paths_at_timestep):
        """Copied verbatim from LSM.py::LeastSquaresPricer.calculate_continuation_value()"""
        if self.train_ITM_only:
            in_the_money     = np.where(immediate_exercise_value[:self.split] > 0)
            in_the_money_all = np.where(immediate_exercise_value > 0)
        else:
            in_the_money     = np.where(immediate_exercise_value[:self.split] < np.inf)
            in_the_money_all = np.where(immediate_exercise_value < np.inf)
        return_values = np.zeros(stock_paths_at_timestep.shape[0])
        return_values[in_the_money_all[0]] = self.regression.calculate_regression(
            stock_paths_at_timestep, values, in_the_money, in_the_money_all)
        return return_values


# Run
lsm_pricer = LeastSquaresPricer(model, payoff, train_ITM_only=True)
t0 = time.time()
price_lsm, _ = lsm_pricer.price(stock_paths, var_paths)
t_lsm = time.time() - t0
print(f'LSM  price={price_lsm:.4f}  ref={ref_price:.4f}  err={price_lsm-ref_price:+.4f}  [{t_lsm:.2f}s]')

## 8 · Algorithm 2 — RLSM

**Source:**
- `algorithms/utils/randomized_neural_networks.py` → `Reservoir2` (not uploaded — inlined from usage in `regression.py`)
- `regression.py` → `ReservoirLeastSquares2.calculate_regression()` — **verbatim** (already used PyTorch)
- `RLSM.py` → `ReservoirLeastSquarePricerFastTanh` — **verbatim**

`Reservoir2` is a `torch.nn.Module` with **frozen random weights** (no gradient updates ever).  
Only the output layer (solved by OLS) is fitted per time step.

In [ ]:
# SOURCE: algorithms/utils/randomized_neural_networks.py → Reservoir2
# NOT UPLOADED — inlined from its usage in regression.py::ReservoirLeastSquares2:
#   self.reservoir = randomized_neural_networks.Reservoir2(
#       hidden_size, self.state_size, factors=factors, activation=activation)
#   self.reservoir(X)   ← called as a nn.Module in calculate_regression()
class Reservoir2(nn.Module):
    """Frozen random single-hidden-layer network (ELM).
    Source: algorithms/utils/randomized_neural_networks.py::Reservoir2 (inlined)
    Weights drawn once, frozen forever. Only OLS output layer changes per step.
    factors[0] controls weight init scale (matches RLSM.py usage: factors=(1.,)).
    """
    def __init__(self, hidden_size, state_size, factors=(1.,), activation=nn.Tanh()):
        super().__init__()
        self.activation = activation
        self.linear     = nn.Linear(state_size, hidden_size)
        self._init_and_freeze(factors)

    def _init_and_freeze(self, factors):
        scale = factors[0] if len(factors) > 0 else 1.0
        nn.init.normal_(self.linear.weight, mean=0.0, std=scale)
        nn.init.uniform_(self.linear.bias, -1.0, 1.0)
        for p in self.parameters():
            p.requires_grad_(False)   # FROZEN — never updated

    def init(self):
        """Re-randomise weights (used when reinit=True in ReservoirLeastSquares2)."""
        self._init_and_freeze((1.,))

    def forward(self, x):
        return self.activation(self.linear(x))


# SOURCE: regression.py → ReservoirLeastSquares2 (verbatim)
# Already used PyTorch in the original — no TF conversion needed.
class ReservoirLeastSquares2:
    """OLS regression on frozen random neural network features.
    Source: regression.py::ReservoirLeastSquares2
    """
    def __init__(self, state_size, hidden_size=10, factors=(1.,),
                 activation=nn.LeakyReLU(0.5), reinit=False):
        self.nb_base_fcts = hidden_size + 1
        self.state_size   = state_size
        self.reinit       = reinit
        self.reservoir    = Reservoir2(
            hidden_size, self.state_size, factors=factors, activation=activation)

    def calculate_regression(self, X_unsorted, Y, in_the_money, in_the_money_all,
                              coefficients=None, return_coefficients=False):
        """Copied verbatim from regression.py::ReservoirLeastSquares2.calculate_regression()"""
        if self.reinit:
            self.reservoir.init()
        X = torch.from_numpy(X_unsorted).type(torch.float32)
        reg_input = np.concatenate(
            [self.reservoir(X).detach().numpy(), np.ones((len(X), 1))], axis=1)
        if coefficients is None:
            coefficients = np.linalg.lstsq(
                reg_input[in_the_money[0]], Y[in_the_money[0]], rcond=None)
        continuation_values = np.dot(reg_input[in_the_money_all[0]], coefficients[0])
        if return_coefficients:
            return continuation_values, coefficients
        return continuation_values


# SOURCE: RLSM.py → ReservoirLeastSquarePricerFastTanh (verbatim)
class ReservoirLeastSquarePricerFastTanh(LeastSquaresPricer):
    """RLSM pricer with Tanh activation on the frozen reservoir.
    Source: RLSM.py::ReservoirLeastSquarePricerFastTanh
    Inherits calculate_continuation_value() from LeastSquaresPricer unchanged;
    only self.regression is swapped to ReservoirLeastSquares2(activation=Tanh).
    """
    def __init__(self, model, payoff, hidden_size=10, factors=(1.,),
                 nb_epochs=None, nb_batches=None, train_ITM_only=True,
                 use_payoff_as_input=False):
        super().__init__(model, payoff, train_ITM_only=train_ITM_only,
                         use_payoff_as_input=use_payoff_as_input)
        if hidden_size < 0:
            hidden_size = 50 + abs(hidden_size) * model.nb_stocks
        state_size = model.nb_stocks * (1 + self.use_var) + self.use_payoff_as_input * 1
        self.regression = ReservoirLeastSquares2(
            state_size, hidden_size, factors=factors, activation=nn.Tanh())


# Run
rlsm_pricer = ReservoirLeastSquarePricerFastTanh(
    model, payoff, hidden_size=100, factors=(1.,), train_ITM_only=True)
t0 = time.time()
price_rlsm, _ = rlsm_pricer.price(stock_paths, var_paths)
t_rlsm = time.time() - t0
print(f'RLSM price={price_rlsm:.4f}  ref={ref_price:.4f}  err={price_rlsm-ref_price:+.4f}  [{t_rlsm:.2f}s]')

## 9 · Algorithm 3 — NLSM

**Source:**
- `algorithms/utils/neural_networks.py` → `NetworkNLSM` (not uploaded) — **TF/Keras → PyTorch conversion**
- `NLSM.py` → `init_weights`, `NeuralRegression`, `NeuralNetworkPricer` — **verbatim**  
  (`train_network` and `evaluate_network` already used PyTorch in the original)

**TF → PyTorch details:**  
The original `NetworkNLSM` was `tf.keras.Sequential([Dense(hidden), Tanh, Dense(hidden), Tanh, Dense(1)])`.  
Replaced with `torch.nn.Sequential` using identical layer sizes and activations.  
Called with `.double()` to match the original Keras float64 default (see `NeuralRegression.__init__`).

In [ ]:
# SOURCE: algorithms/utils/neural_networks.py → NetworkNLSM
# NOT UPLOADED. Original was TensorFlow/Keras. CONVERTED TO PyTorch.
# Architecture unchanged: Linear→Tanh→Linear→Tanh→Linear(1)
class NetworkNLSM(nn.Module):
    """MLP for NLSM continuation value.
    Source: algorithms/utils/neural_networks.py::NetworkNLSM
    CONVERTED from TensorFlow/Keras to PyTorch.
    Architecture unchanged: two hidden layers with Tanh, output dim=1.
    Called as .double() in NeuralRegression to match original Keras float64.
    """
    def __init__(self, nb_stocks, hidden_size=10):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(nb_stocks, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1)
        )

    def forward(self, x):
        return self.net(x)   # (N, 1)


# SOURCE: NLSM.py → init_weights (verbatim)
def init_weights(m):
    """Xavier uniform init. Source: NLSM.py::init_weights"""
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)


# SOURCE: NLSM.py → NeuralRegression (verbatim)
# train_network() and evaluate_network() already used PyTorch in the original.
# Only change: NetworkNLSM is now the PyTorch version above.
class NeuralRegression:
    """Train/evaluate the MLP for the NLSM continuation value.
    Source: NLSM.py::NeuralRegression (verbatim)
    """
    def __init__(self, nb_stocks, nb_paths, hidden_size=10,
                 nb_iters=20, batch_size=2000):
        self.batch_size     = batch_size
        self.nb_stocks      = nb_stocks
        self.nb_paths       = nb_paths
        self.nb_iters       = nb_iters
        # .double() matches original Keras float64 weights
        self.neural_network = NetworkNLSM(self.nb_stocks, hidden_size=hidden_size).double()
        self.neural_network.apply(init_weights)

    def train_network(self, X_inputs, Y_labels):
        """Copied verbatim from NLSM.py::NeuralRegression.train_network()"""
        optimizer = optim.Adam(self.neural_network.parameters())
        X_inputs  = torch.from_numpy(X_inputs).double()
        Y_labels  = torch.from_numpy(Y_labels).double().view(len(Y_labels), 1)
        self.neural_network.train(True)
        for iteration in range(self.nb_iters):
            for batch in tdata.BatchSampler(
                    tdata.RandomSampler(range(len(X_inputs)), replacement=False),
                    batch_size=self.batch_size, drop_last=False):
                optimizer.zero_grad()
                with torch.set_grad_enabled(True):
                    outputs = self.neural_network(X_inputs[batch])
                    loss    = nn.MSELoss(reduction='mean')(outputs, Y_labels[batch])
                    loss.backward()
                    optimizer.step()

    def evaluate_network(self, X_inputs):
        """Copied verbatim from NLSM.py::NeuralRegression.evaluate_network()"""
        self.neural_network.train(False)
        X_inputs = torch.from_numpy(X_inputs).double()
        outputs  = self.neural_network(X_inputs)
        return outputs.view(len(X_inputs)).detach().numpy()


# SOURCE: NLSM.py → NeuralNetworkPricer (verbatim)
class NeuralNetworkPricer(AmericanOptionPricer):
    """American option pricer — Neural Least-Square Monte Carlo.
    Source: NLSM.py::NeuralNetworkPricer (verbatim)
    The only change: NeuralRegression uses PyTorch NetworkNLSM instead of Keras.
    """
    def __init__(self, model, payoff, nb_epochs=20, nb_batches=None,
                 hidden_size=10, train_ITM_only=True, use_payoff_as_input=False):
        del nb_batches
        super().__init__(model, payoff, train_ITM_only=train_ITM_only,
                         use_payoff_as_input=use_payoff_as_input)
        self.neural_regression = NeuralRegression(
            model.nb_stocks * (1 + self.use_var) + self.use_payoff_as_input * 1,
            model.nb_paths, hidden_size=hidden_size, nb_iters=nb_epochs)

    def calculate_continuation_value(
            self, values, immediate_exercise_value, stock_paths_at_timestep):
        """Copied verbatim from NLSM.py::NeuralNetworkPricer.calculate_continuation_value()"""
        inputs = stock_paths_at_timestep
        if self.train_ITM_only:
            in_the_money     = np.where(immediate_exercise_value[:self.split] > 0)
            in_the_money_all = np.where(immediate_exercise_value > 0)
        else:
            in_the_money     = np.where(immediate_exercise_value[:self.split] < np.inf)
            in_the_money_all = np.where(immediate_exercise_value < np.inf)
        continuation_values = np.zeros(stock_paths_at_timestep.shape[0])
        self.neural_regression.train_network(
            inputs[in_the_money[0]], values[in_the_money[0]])
        continuation_values[in_the_money_all[0]] = \
            self.neural_regression.evaluate_network(inputs[in_the_money_all[0]])
        return continuation_values


# Run
nlsm_pricer = NeuralNetworkPricer(
    model, payoff, nb_epochs=20, hidden_size=50, train_ITM_only=True)
t0 = time.time()
price_nlsm, _ = nlsm_pricer.price(stock_paths, var_paths)
t_nlsm = time.time() - t0
print(f'NLSM price={price_nlsm:.4f}  ref={ref_price:.4f}  err={price_nlsm-ref_price:+.4f}  [{t_nlsm:.2f}s]')

## 10 · Results Summary

In [ ]:
results = {
    'CRR Binomial (ref)': dict(price=ref_price,  time=None),
    'LSM':                dict(price=price_lsm,  time=t_lsm),
    'RLSM':               dict(price=price_rlsm, time=t_rlsm),
    'NLSM':               dict(price=price_nlsm, time=t_nlsm),
}
print(f"\n{'='*62}")
print(f"  American Put  S0={SPOT} K={STRIKE} r={RATE} σ={VOLATILITY} T={MATURITY}")
print(f"  Paths={NB_PATHS:,}   Dates={NB_DATES}")
print(f"{'='*62}")
print(f"  {'Algorithm':<22} {'Price':>8}  {'Error':>8}  {'Time':>8}")
print(f"  {'-'*52}")
for name, v in results.items():
    err   = f"{v['price']-ref_price:+.4f}" if v['time'] is not None else '   ref  '
    t_str = f"{v['time']:.2f}s"            if v['time'] is not None else '   —    '
    print(f"  {name:<22} {v['price']:>8.4f}  {err:>8}  {t_str:>8}")
print(f"{'='*62}")

In [ ]:
algos  = ['LSM', 'RLSM', 'NLSM']
prices = [price_lsm, price_rlsm, price_nlsm]
times  = [t_lsm,     t_rlsm,     t_nlsm]
colors = ['#2196F3', '#4CAF50',  '#FF9800']

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
ax = axes[0]
bars = ax.bar(algos, prices, color=colors, width=0.5, edgecolor='black', lw=0.8)
ax.axhline(ref_price, color='red', lw=2, ls='--', label=f'CRR ref={ref_price:.4f}')
ax.set_ylim(ref_price*0.97, ref_price*1.03)
for bar, p in zip(bars, prices):
    ax.text(bar.get_x()+bar.get_width()/2, p+0.002, f'{p:.4f}',
            ha='center', va='bottom', fontsize=11, fontweight='bold')
ax.set_title('Price Estimate vs Reference', fontsize=13)
ax.set_ylabel('Price'); ax.legend(fontsize=11); ax.grid(axis='y', alpha=0.4)
ax = axes[1]
bars2 = ax.bar(algos, times, color=colors, width=0.5, edgecolor='black', lw=0.8)
for bar, t in zip(bars2, times):
    ax.text(bar.get_x()+bar.get_width()/2, t*1.02, f'{t:.2f}s',
            ha='center', va='bottom', fontsize=11, fontweight='bold')
ax.set_title('Wall-Clock Runtime', fontsize=13)
ax.set_ylabel('Seconds'); ax.grid(axis='y', alpha=0.4)
plt.suptitle(f'American Put  S0={SPOT} K={STRIKE} σ={VOLATILITY} r={RATE} T={MATURITY}'
             f'  |  {NB_PATHS:,} paths · {NB_DATES} dates', fontsize=11, y=1.01)
plt.tight_layout(); plt.show()

## 11 · Sensitivity — Price Convergence vs Number of Paths

In [ ]:
path_counts  = [2_000, 5_000, 10_000, 25_000, 50_000]
conv_results = {a: [] for a in algos}

for n in path_counts:
    print(f'n={n:>6,}', end='  ')
    m_n = BlackScholes(drift=RATE, volatility=VOLATILITY, nb_paths=n,
                       nb_stocks=1, nb_dates=NB_DATES, spot=SPOT,
                       maturity=MATURITY, dividend=DIVIDEND)
    np.random.seed(SEED)
    sp_n, vp_n = m_n.generate_paths()

    p_lsm,  _ = LeastSquaresPricer(m_n, payoff).price(sp_n, vp_n)
    p_rlsm, _ = ReservoirLeastSquarePricerFastTanh(
                    m_n, payoff, hidden_size=100).price(sp_n, vp_n)
    p_nlsm, _ = NeuralNetworkPricer(
                    m_n, payoff, nb_epochs=20, hidden_size=50).price(sp_n, vp_n)
    conv_results['LSM'].append(p_lsm)
    conv_results['RLSM'].append(p_rlsm)
    conv_results['NLSM'].append(p_nlsm)
    print(f'LSM={p_lsm:.4f}  RLSM={p_rlsm:.4f}  NLSM={p_nlsm:.4f}')

fig, ax = plt.subplots(figsize=(9, 5))
for alg, col in zip(algos, colors):
    ax.plot(path_counts, conv_results[alg], 'o-', color=col, lw=2, ms=7, label=alg)
ax.axhline(ref_price, color='red', lw=2, ls='--', label=f'CRR ref={ref_price:.4f}')
ax.set_xscale('log')
ax.set_xlabel('Number of Paths (log scale)', fontsize=12)
ax.set_ylabel('Price Estimate', fontsize=12)
ax.set_title('Convergence vs Number of Paths', fontsize=13)
ax.legend(fontsize=11); ax.grid(alpha=0.4)
plt.tight_layout(); plt.show()

## 12 · Sensitivity — RLSM Reservoir Size

A key finding of Herrera et al. (2023): RLSM price quality improves rapidly with more random neurons.  
The cost increase is minimal — there is no backprop, only a larger closed-form OLS solve.

In [ ]:
hidden_sizes  = [10, 25, 50, 100, 200, 400]
rlsm_prices_h = []
rlsm_times_h  = []

for h in hidden_sizes:
    pr = ReservoirLeastSquarePricerFastTanh(model, payoff, hidden_size=h)
    t0 = time.time()
    p, _ = pr.price(stock_paths, var_paths)
    t_   = time.time() - t0
    rlsm_prices_h.append(p); rlsm_times_h.append(t_)
    print(f'  H={h:4d}  price={p:.4f}  err={p-ref_price:+.4f}  [{t_:.2f}s]')

fig, axes = plt.subplots(1, 2, figsize=(13, 5))
ax = axes[0]
ax.plot(hidden_sizes, rlsm_prices_h, 'o-', color='#4CAF50', lw=2, ms=8)
ax.axhline(ref_price, color='red', lw=2, ls='--', label=f'CRR ref={ref_price:.4f}')
ax.set_xlabel('Hidden Units (H)', fontsize=12); ax.set_ylabel('Price', fontsize=12)
ax.set_title('RLSM: Accuracy vs Reservoir Size', fontsize=13)
ax.legend(fontsize=11); ax.grid(alpha=0.4)
ax = axes[1]
ax.plot(hidden_sizes, rlsm_times_h, 's-', color='#4CAF50', lw=2, ms=8)
ax.set_xlabel('Hidden Units (H)', fontsize=12); ax.set_ylabel('Seconds', fontsize=12)
ax.set_title('RLSM: Runtime vs Reservoir Size', fontsize=13)
ax.grid(alpha=0.4)
plt.tight_layout(); plt.show()